In [1]:
from core.meta import meta_interface
%matplotlib inline

In [2]:
d = meta_interface(project_config_path="test_data/project_config.yaml")
# manually add recording_configs 
d.add_recording_config('test_data/Server_structure/Calibrations/220826/recording_config.yaml')
d.initialize_meta_config()
#d.meta
# switch recordings to other Recording_days -> calibration_directroy

In [3]:
d.export_meta_to_yaml(filename = "test_data/meta_test.yaml")
# change recordingvideo target_fps, fps in videos, filepaths (e.g. if multiple topcam files)

In [4]:
d.load_meta_from_yaml(filename = "test_data/meta_test.yaml")

In [5]:
d.create_recordings()
#d.meta

In [6]:
d.export_meta_to_yaml(filename = "test_data/meta_test.yaml")
# change recordingvideo target_fps, fps in videos, filepaths (e.g. if multiple topcam files)

In [7]:
d.load_meta_from_yaml(filename = "test_data/meta_test.yaml")

In [8]:
d.synchronize_recordings()
#d.meta
# exclude recordingvideos based on synchronization

KeyboardInterrupt: 

In [10]:
d.create_calibrations()
#d.meta

In [ ]:
d.export_meta_to_yaml(filename = "test_data/meta_test.yaml")
# change calibrationvideo target_fps, fps in videos, filepaths (e.g. if multiple topcam files)

In [ ]:
d.load_meta_from_yaml(filename = "test_data/meta_test.yaml")

In [ ]:
d.synchronize_calibrations()
#d.meta
# exclude calibrationvideos based on synchronization, change markerdetection_filepaths etc.

In [ ]:
import yaml
with open("test_data/Server_structure/Calibrations/220826/positionsh5s.yaml", "r") as ymlfile:
    positionsh5s = yaml.load(ymlfile, Loader=yaml.SafeLoader)
for video in d.meta["recording_days"]["Recording_Day_220826_0"]["calibrations"]["videos"].values():
    video["positions_marker_detection_filepath"]=positionsh5s[video.cam_id]

In [ ]:
d.export_meta_to_yaml(filename = "test_data/meta_test.yaml")

In [ ]:
d.load_meta_from_yaml(filename = "test_data/meta_test.yaml")

In [ ]:
d.calibrate()
#d.meta
# exclude calibrations based on calibration log (not yet implemented) and repeat calibration 

In [16]:
d.export_meta_to_yaml(filename = "test_data/meta_test.yaml")

In [17]:
d.load_meta_from_yaml(filename = "test_data/meta_test.yaml")

In [18]:
d.triangulate_recordings()

In [ ]:
Was geschehen soll:
    
1.
- alle recording configs manuell hinzufügen
- alle recording directories finden/manuell hinzufügen
- Recording_Triangulation objekte erstellen
# in notebook (per user input) filenaming (paradigm, etc.) korrigieren
- metadata sammeln
STOP 
2.
- synchronisiert videos und läuft DLC
- meta: synchro_path, DLC_path, synchro_framenum
STOP: check exclusion manually!
3.
?A
- necessary_calibrations based on exclusion_flags
- meta: calibration_to_use based on necessary_calibrations
- create calibration objects based on necessary_calibrations
4.
Calibration wird erstellt
- meta: videos mit cam_id, fps, framenum, path, exclusion_flag, intrinsic_calibrations
erstellt Positions
- meta: Positions
STOP
5.
- Positions: marker predicition
- synchronisiert videos
- meta: synchro videos, marker_predictions positions
STOP
6.
!wenn calbration video excluded wird, repeat von ?A mit overwrite=False, remove video from Triangulation object
- create calibration.toml 
7.
Triangulation_Recordings
- run triangulation
- meta: 3D.csv, error

In [ ]:
Was bisher noch fehlt: 

code schöner: 
- go through.triang_calib_module, video_module, video_synchronization_module
- remove already synchronized?
- black

update intrinsic calibrations in all recording video and calibration video objects?
6. (sobald implementiert: optimize calibration based on positions dafür: triangulate positions) 

In [ ]:
how to handle:
    several calibrations at one day:
        individual directories and recording_configs per calibration, set calibration_index in recording_configs individually!
    in meta.yaml: all recordings of the day can be found at all calibrations of the day -> just keep the right recordings and delete the others

In [ ]:
from core.triangulation_calibration_module import Calibration, Triangulation_Positions, Triangulation_Recordings

In [ ]:
a = Calibration(calibration_directory = 'test_data/Server_structure/Calibrations/220826/', 
                project_config_filepath = 'test_data/project_config.yaml',
                recording_config_filepath = 'test_data/Server_structure/Calibrations/220826/recording_config.yaml',
                output_directory = 'test_data/Server_structure/Calibrations/220826/')

In [ ]:
a.run_synchronization()

In [ ]:
a.run_calibration(verbose = True)

In [ ]:
b = Triangulation_Recordings(recording_directory = 'test_data/Server_structure/VGlut2-flp/September2022/196_F7-27/220826_OTT/', 
                             calibration_directory = 'test_data/Server_structure/Calibrations/220826/',
                             recording_config_filepath = 'test_data/Server_structure/Calibrations/220826/recording_config.yaml', 
                             project_config_filepath= 'test_data/project_config.yaml',
                             output_directory = "test_data/Server_structure/VGlut2-flp/September2022/196_F7-27/220826_OTT/")

In [ ]:
b.run_synchronization()

In [ ]:
b.run_triangulation(calibration_toml_filepath = 'test_data/Server_structure/Calibrations/220826/calibration_220826.toml')

In [ ]:
c = Triangulation_Positions(positions_directory = 'test_data/Server_structure/Calibrations/220826/', 
                            calibration_directory = 'test_data/Server_structure/Calibrations/220826/',
                            recording_config_filepath = 'test_data/Server_structure/Calibrations/220826/recording_config.yaml', 
                            project_config_filepath= 'test_data/project_config.yaml',
                            output_directory = 'test_data/Server_structure/Calibrations/220826/')

In [ ]:
c.get_marker_predictions()

In [ ]:
c.run_triangulation(calibration_toml_filepath = 'test_data/Server_structure/Calibrations/220826/calibration_220826.toml')

In [ ]:
# special dependencies

In [ ]:
"""
pip install aniposelib
pip install ffmpeg
pip install ffmpeg-python
pip install imageio-ffmpeg
jupyter lab
Tkinter
"""